In [2]:
import pathlib
from bs4 import BeautifulSoup
from IPython.display import clear_output
import pandas as pd
import datetime
import pickle

In [3]:
root = pathlib.Path('D:\股票網頁\股票\價量')
exchange = root.joinpath('上市')

In [18]:
exchange = sorted(exchange.glob('*.html'))

In [19]:
for html in  exchange:
    print(html.as_posix())
    table =  html.read_text('utf8')
    if '很抱歉，沒有符合條件的資料!' in table:
        clear_output(wait=True)
        continue
    else:
        table = BeautifulSoup(table)
        table = [x for x in table.find_all('table') if '每日收盤行情(全部(不含權證、牛熊證))' in x.text][0]
        table = pd.read_html(table.prettify(),encoding='utf8',converters={'證券代號':str})[0]
        
        title_date = table.columns.levels[0][0]
        date = datetime.datetime.strptime(html.name,'%Y-%m-%d.html')
        date = date.strftime(f'{date.year-1911:>03}年%m月%d日每日收盤行情(全部(不含權證、牛熊證))')
        assert date == title_date

        table.columns = [col[-1] for col in table.columns]
        assert table.columns.tolist() == ['證券代號','證券名稱',
                                         '成交股數','成交筆數','成交金額','開盤價','最高價','最低價','收盤價',
                                         '漲跌(+/-)','漲跌價差','最後揭示買價','最後揭示買量','最後揭示賣價','最後揭示賣量','本益比']
        table.to_csv(root.joinpath('上市_csv',html.with_suffix('.csv').name),encoding='utf8',index=False)
        clear_output(wait=True)

D:/股票網頁/價量/上市/2021-05-26.html


In [28]:
with open('BusinessDates.idx','rb') as f:
    business_dates = pickle.load(f)

In [29]:
business_dates = business_dates[(business_dates>='2004-02-11') & (business_dates<='2021-05-26')]

In [30]:
business_dates

DatetimeIndex(['2004-02-11', '2004-02-12', '2004-02-13', '2004-02-16',
               '2004-02-17', '2004-02-18', '2004-02-19', '2004-02-20',
               '2004-02-23', '2004-02-24',
               ...
               '2021-05-13', '2021-05-14', '2021-05-17', '2021-05-18',
               '2021-05-19', '2021-05-20', '2021-05-21', '2021-05-24',
               '2021-05-25', '2021-05-26'],
              dtype='datetime64[ns]', length=4272, freq=None)

In [31]:
market_csv = sorted(x.with_suffix('').name for x in root.joinpath('上市_csv').glob('*.csv'))

In [32]:
market_csv = pd.DatetimeIndex(market_csv)

In [33]:
market_csv

DatetimeIndex(['2004-02-11', '2004-02-12', '2004-02-13', '2004-02-16',
               '2004-02-17', '2004-02-18', '2004-02-19', '2004-02-20',
               '2004-02-23', '2004-02-24',
               ...
               '2021-05-13', '2021-05-14', '2021-05-17', '2021-05-18',
               '2021-05-19', '2021-05-20', '2021-05-21', '2021-05-24',
               '2021-05-25', '2021-05-26'],
              dtype='datetime64[ns]', length=4272, freq=None)

In [34]:
market_csv.difference(business_dates)

DatetimeIndex([], dtype='datetime64[ns]', freq=None)

In [35]:
business_dates.difference(market_csv)

DatetimeIndex([], dtype='datetime64[ns]', freq=None)

In [36]:
(business_dates == market_csv).all()

True

In [38]:
data = pd.DataFrame()
for csv in root.joinpath('上市_csv').glob('*.csv'):
    print(csv.as_posix())
    table = pd.read_csv(csv,encoding='utf8',dtype={'證券代號':str})
    table['日期'] = csv.with_suffix('').name
    data = data.append(table)
    clear_output(wait=True)

D:/股票網頁/價量/上市_csv/2021-05-26.csv


In [41]:
# data.to_csv(root.joinpath('上市.csv'),encoding='utf8',index=False)

In [4]:
data = pd.read_csv(root.joinpath('上市.csv'),encoding='utf8',index_col='證券代號',dtype=str)

In [5]:
data

,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,日期
證券代號,,,,,,,,,,,,,,,,
0001,鴻運,19000,7,147700,7.70,7.85,7.70,7.70,-,0.15,7.70,NaN,7.75,NaN,0.0,2004-02-11
0015,富邦,301000,40,1851900,6.20,6.20,6.15,6.15,-,0.05,6.10,NaN,6.15,NaN,0.0,2004-02-11
0029,富邦店,365000,63,1796740,4.95,4.95,4.89,4.89,-,0.07,4.89,NaN,4.92,NaN,0.0,2004-02-11
0050,元大台灣50,5264000,686,254721750,48.50,48.50,48.26,48.37,-,0.17,48.30,NaN,48.37,NaN,0.0,2004-02-11
1101,台泥,44990547,5708,875661311,19.90,19.90,19.10,19.30,-,0.4,19.30,NaN,19.40,NaN,24.74,2004-02-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9944,新麗,793685,398,17404328,21.90,22.20,21.60,21.95,+,0.35,21.9,4.0,21.95,44.0,29.66,2021-05-26
9945,潤泰新,2819141,1536,145623343,51.70,51.90,51.30,51.80,+,0.1,51.8,64.0,51.90,110.0,6.36,2021-05-26
9946,三發地產,303874,146,4612476,14.90,15.30,14.90,15.30,+,0.45,15.25,2.0,15.30,30.0,95.62,2021-05-26


In [6]:
data = data.loc[:,['日期','證券名稱','成交股數','成交筆數','成交金額','開盤價','最高價','最低價','收盤價']]

In [20]:
def abnormal_filter(row):
    try:
        row.astype(float)
        return False
    except ValueError:
        return True

In [23]:
data.loc[(data[['成交股數','成交筆數','成交金額','開盤價','最高價','最低價','收盤價']]=='--').any(axis=1)]

C:\Users\user\anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,日期,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價
證券代號,,,,,,,,,
1101A,2004-02-11,台泥特,0,0,0,--,--,--,--
16051,2004-02-11,華新一,0,0,0,--,--,--,--
1618R,2004-02-11,合機甲,0,0,0,--,--,--,--
17141,2004-02-11,和桐一,0,0,0,--,--,--,--
17142,2004-02-11,和桐二,0,0,0,--,--,--,--
...,...,...,...,...,...,...,...,...,...
00707R,2021-04-27,期元大S&P日圓反1,0,0,0,--,--,--,--
00774C,2021-04-27,新光中政金綠債+R,0,0,0,--,--,--,--
020004,2021-04-27,兆豐電菁英30N,0,0,0,--,--,--,--


In [7]:
data.dtypes

日期      object
證券名稱    object
成交股數    object
成交筆數    object
成交金額    object
開盤價     object
最高價     object
最低價     object
收盤價     object
dtype: object

In [10]:
data

,日期,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價
證券代號,,,,,,,,,
0001,2004-02-11,鴻運,19000,7,147700,7.7,7.85,7.7,7.7
0015,2004-02-11,富邦,301000,40,1851900,6.2,6.2,6.15,6.15
0029,2004-02-11,富邦店,365000,63,1796740,4.95,4.95,4.89,4.89
0050,2004-02-11,元大台灣50,5264000,686,254721750,48.5,48.5,48.26,48.37
1101,2004-02-11,台泥,44990547,5708,875661311,19.9,19.9,19.1,19.3
...,...,...,...,...,...,...,...,...,...
9944,2021-05-26,新麗,793685,398,17404328,21.9,22.2,21.6,21.95
9945,2021-05-26,潤泰新,2819141,1536,145623343,51.7,51.9,51.3,51.8
9946,2021-05-26,三發地產,303874,146,4612476,14.9,15.3,14.9,15.3


In [14]:
data = data.loc[data['開盤價']!='--']
data[['開盤價','最高價','最低價','收盤價']] = data[['開盤價','最高價','最低價','收盤價']].astype(float)

In [16]:
data[['成交股數','成交筆數','成交金額']] = data[['成交股數','成交筆數','成交金額']].astype('int64')

In [17]:
data.dtypes

日期       object
證券名稱     object
成交股數      int64
成交筆數      int64
成交金額      int64
開盤價     float64
最高價     float64
最低價     float64
收盤價     float64
dtype: object

In [18]:
ohlc = data.loc[:,['開盤價','最高價','最低價','收盤價']]
high = ohlc.max(axis=1)
low = ohlc.min(axis=1)

In [19]:
data.loc[ohlc['最高價'] != high]

,日期,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價
證券代號,,,,,,,,,


In [20]:
data.loc[ohlc['最低價'] != low]

,日期,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價
證券代號,,,,,,,,,


In [21]:
data.to_csv(root.joinpath('上市_final.csv'),encoding='utf8')